# How to crawling?

In [1]:
#import custom module
import havester as hv
import pandas as pd
from IPython.display import clear_output 

hv.IS_LOGGING = False

## 위치을 이용하여 구역 특정

In [3]:
# loc = hv.NLocation(35.18213, 128.1366) # 진주 충무공동
loc = hv.NLocation(37.49911, 127.065463) # 강남 대치동

cur_region = hv.NRegion('', hv.NLocation(37.49911, 127.065463), '0000000000')

# 검색해서 지역 찾기
while True:
    regions = hv.get_region_list(cur_region.no)
    if len(regions) == 0: break
    for i, region in enumerate(regions): 
        print(i, region)
        
    idx = int(input("원하는 도시 인덱스(exit -1) : "))
    if idx == -1: break
    cur_region = regions[idx]
    clear_output()
    
sector = hv.get_sector(cur_region.loc)

print(sector)

서울시 강남구 신사동 1168010700 loc(37.524142 | 127.022900)


## 매물 조건 생성

In [4]:
addon = hv.NAddon(
    tradeType=[hv.NAddon.TRADE_DEAL, hv.NAddon.TRADE_LEASE], #목표 거래 - 매매, 전세
    estateType=[hv.NAddon.ESTATE_APT, hv.NAddon.ESTATE_OPST] #목표 매물 - 아파트, 오피스텔
)

## 조건과 구역을 이용하여 매물 탐색

In [5]:
things = [] # 매물 기록
for dirr in hv.NAddon.DIR_EACH: # 모든 방향 (남향 등등)
    addon.direction = [dirr] # 매물 방향 선택
    res = hv.get_things(sector, addon)
    
    for v in res:
        v.dir = dirr # 매물 방향 정의
    things.extend(res)
print('매물 수', len(things))

매물 수 328


## 구역 주변 편의 시설 탐색

In [6]:
neighbors = [] # 편의시설 기록
for nType in hv.NNeighbor.EACH: # 모든 편의시설
    res = hv.get_neighborhood(sector.loc, nType)
    neighbors.extend(res)
    print(nType, len(res))
print('Total', len(neighbors))

BUS 38
METRO 1
INFANT 13
PRESCHOOL 1
SCHOOLPOI 25
HOSPITAL 0
PARKING 125
MART 4
CONVENIENCE 80
WASHING 37
BANK 49
OFFICE 11
Total 384


# 편의시설과 매물 간의 거리 탐색

In [7]:
# 500 == 걸어서 5~10분 이내, 버스로 1분
# 1000 == 버스로 1~2분 

standard = hv.get_distance_standard() # 거리 기준

for (key, value) in standard.items():
    print(key, value)

BUS 500
METRO 500
INFANT 750
PRESCHOOL 750
SCHOOLPOI 2000
HOSPITAL 5000
PARKING 500
MART 500
CONVENIENCE 250
WASHING 500
BANK 1000
OFFICE 2000


In [8]:
for thing in things: # 매물
    around = hv.NNeighborAround()
    for nei in neighbors: # 편의시설
        d = hv.distance_between(thing.loc, nei.loc)
        if d <= standard[nei.type]: # meter
            around.increase(nei.type)
    thing.neiAround = around

## 시각화

In [9]:
thing_list = []
around_nei = []

for t in things: 
    thing_list.append(t.get_list())
    around_nei.append(t.neiAround.get_list())

thing_df = pd.DataFrame(data=thing_list ,columns=hv.NThing.HEADER)
nei_df = pd.DataFrame(data=around_nei, columns=hv.NNeighborAround.HEADER)
total_df = pd.concat([thing_df, nei_df], axis=1)



total_df.head(5)

,Name,Type,Build,Dir,minArea,maxArea,representativeArea,floorAreaRatio,minDeal,maxDeal,...,INFANT,PRESCHOOL,SCHOOLPOI,HOSPITAL,PARKING,MART,CONVENIENCE,WASHING,BANK,OFFICE
0,현대빌라트,아파트,199608,EE,203.03,203.03,203.0,237,580000,620000,...,2,0,25,0,13,0,0,3,34,11
1,현대썬앤빌(도시형),아파트,201502,EE,27.74,27.74,27.0,799,0,0,...,9,1,25,0,42,1,8,11,45,11
2,금광아크존(주상복합),아파트,200409,EE,86.38,86.38,86.0,279,140000,145000,...,5,0,25,0,27,0,6,4,39,11
3,현대썬앤빌,오피스텔,201502,EE,33.12,35.21,33.0,799,0,0,...,9,1,25,0,42,1,8,11,45,11
4,신반포16차,아파트,198301,EE,57.58,91.49,57.0,0,189000,270000,...,3,0,12,0,1,0,0,2,4,2


In [10]:
nei_df.describe() # 편의시설 접근도

,BUS,METRO,INFANT,PRESCHOOL,SCHOOLPOI,HOSPITAL,PARKING,MART,CONVENIENCE,WASHING,BANK,OFFICE
count,328.00000,328.000000,328.000000,328.000000,328.000000,328.0,328.000000,328.000000,328.000000,328.000000,328.000000,328.000000
mean,7.97561,0.268293,5.219512,0.512195,23.609756,0.0,26.243902,0.756098,4.902439,7.707317,28.658537,10.097561
std,3.52874,0.443748,2.468267,0.500615,3.404462,0.0,16.481710,0.878712,4.372865,4.882692,14.590198,2.390653
min,2.00000,0.000000,1.000000,0.000000,12.000000,0.0,1.000000,0.000000,0.000000,2.000000,4.000000,2.000000
25%,5.00000,0.000000,3.000000,0.000000,25.000000,0.0,12.000000,0.000000,1.000000,3.000000,12.000000,11.000000
50%,7.00000,0.000000,5.000000,1.000000,25.000000,0.0,25.000000,1.000000,3.000000,6.000000,34.000000,11.000000
75%,10.00000,1.000000,7.000000,1.000000,25.000000,0.0,40.000000,1.000000,8.000000,11.000000,43.000000,11.000000
max,15.00000,1.000000,10.000000,1.000000,25.000000,0.0,57.000000,3.000000,15.000000,20.000000,47.000000,11.000000


## Dataframe 저장

In [11]:
total_df.to_csv('./house_price_data.csv', sep=',', na_rep='NaN')